In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/FieldNN


# Batch

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P-InfoGrn',  'P-EC-PNSect-TknzGrn']

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P-InfoGrn_wgt torch.Size([4, 43])
B-P-InfoGrn_tknidx torch.Size([4, 43])
B-P-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


# get df_SubUnit

In [3]:
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]
full_recfldgrn_list

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

In [4]:
# to fieldnn.dataflowfn.embedflowfn.py

import pandas as pd

def get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunit_name = 'E'):
    SubUnit_List = []
    for input_name in full_recfldgrn_list:
        d = {}
        d['SubUnitName'] = default_E_subunit_name
        recfldgrn = '-'.join(input_name.split('-')[-2:])
        output_layerid = len(input_name.split('-'))
        output_name = input_name.split('Grn')[0]
        d['input_names'] = [input_name]
        d['output_name'] = output_name
        d['output_layerid'] = output_layerid
        SubUnit_List.append(d)
        
    df_SubUnit = pd.DataFrame(SubUnit_List)
    
    return df_SubUnit


In [5]:
default_E_subunit_name = 'E'
df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_E_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid
0,E,[B-P-InfoGrn],B-P-Info,3
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5


# get df_SubUnit Extra Info

## Get SubUnit Name to NN List

* E --> [E]
* M --> [M]


In [6]:
# from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList

def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tkn' in input_name.split('-')[-1]:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            # print(input_names)
            assert idx != 0
            # assert len(input_names) == 1; not necessary, L can follow M
            nn_type = 'learner'
            input_name = input_names[0]
            layers_num = len(input_name.split('-'))
            if layers_num >= 3:
                nn_name = 'TFM'
            elif layers_num == 2:
                nn_name = 'Linear'
            else:
                raise ValueError(f'incorrect layers_num {layers_num}')
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

In [7]:
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'ReduceMax',
    'merger': 'MergeConcat',
    'learner': None, # TODO: ignore this currently
}
#############################################

In [8]:
s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed]
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed]


## Get Default BasicNN Config List

In [9]:
from fieldnn.configfn.expanderfn import get_expander_para

In [10]:
# to dataflowfn.embedflowfn
def get_Default_ExpanderNNPara(full_recfldgrn, fldgrn_folder):
    
    # (1) get basic information
    # recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    recfld = [i for i in full_recfldgrn.split('-')][-2]
    # rec, fld = recfld.split('@')
    # grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    # grn, suffix = grn_suffix.split('_')
    # prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    # recfldgrn = rec + '@' + fld + '-' + grn

    # (2) get vocab information
    # fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, full_recfldgrn[2:] + '.p')
    Info = pd.read_pickle(fullfldgrn_file)


    # (3) get vocab information
    # no matter what type of grain, in the end, we will have vocab_tokenizer.
    # if 'LLM' in full_recfldgrn:
    #     vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    #     init = vocab_tokenizer.name_or_path
    # else:
    #     vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
    #     init = 'random'

    d = {'full_recfldgrn': full_recfldgrn, 'Info': Info}
    return d

In [11]:
def get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                    fldgrn_folder, learner_default_dict):
    
    SubUnit_DefaultBasicNN_List = []
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):
        nn_type, nn_name = nn_type_nn_name.split('-')
        
        if basic_nn_idx == 0:
            # the first BasicNN in the SubUnit
            # expander and merger will only be here.
            # this assigments only works for the first iteration
            # let's make a village rule: only E, R, M can be the first. 
            assert nn_type in ['expander', 'reducer', 'merger']
        
            if nn_type == 'expander':
                input_names_nnlvl = SubUnit_input_names 
                assert len(input_names_nnlvl) == 1
                full_recfldgrn = input_names_nnlvl[0]
                default_para = get_Default_ExpanderNNPara(full_recfldgrn, fldgrn_folder)
                
            else:
                default_para = {}
            
        else:
            assert nn_type in ['reducer', 'learner']
            if nn_type == 'learner':
                default_para = learner_default_dict[nn_name] # TODO
            else:
                default_para = {}
        
        SubUnit_DefaultBasicNN_List.append(default_para)    
        
    return SubUnit_DefaultBasicNN_List

In [12]:
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed]
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed]


In [13]:
SubUnit_info = df_SubUnit.iloc[0]

In [14]:
SubUnit_BasicNN_List = SubUnit_info['SubUnit_BasicNN_List']
SubUnit_input_names = SubUnit_info['input_names']
SubUnit_output_name = SubUnit_info['output_name']
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 

In [15]:
SubUnit_BasicNN_List

['expander-CateEmbed']

In [16]:
SubUnit_DefaultBasicNN_List = get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                                              fldgrn_folder, learner_default_dict)

for i in SubUnit_DefaultBasicNN_List:
    print(i.keys())


dict_keys(['full_recfldgrn', 'Info'])


In [17]:

fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s


df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P..."
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed],"[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '..."


## Get BasicNN Config List

In [18]:
from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para   # ignore this one in this notebook
from fieldnn.configfn.reducerfn import get_reducer_para # ignore this one in this notebook
from fieldnn.configfn.learnerfn import get_learner_para # ignore this one in this notebook


def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        # fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        full_recfldgrn = default_nnpara['full_recfldgrn']
        Info = default_nnpara['Info']
        # para = get_expander_para(nn_name, default_nnpara, embed_size, vocab_tokenizer, init, postprocess)
        para = get_expander_para(full_recfldgrn, Info, embed_size, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1C@DT', 'A1C@V']
        
        fld_childflds = '&'.join([i.split('@')[-1] for i in childflds])
        # fld_childflds: DT&V
        
        output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        # output_prefix: B-P-EC-A1C
        
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
        
    elif nn_type == 'learner':
        # generate the output name
        assert len(input_names_nnlvl) == 1
        output_name_nnlvl = input_names_nnlvl[0] # just the same name as before
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_learner_para(nn_name, default_nnpara, input_size, output_size, postprocess)
        
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config

In [19]:

from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para
from fieldnn.configfn.reducerfn import get_reducer_para
from fieldnn.configfn.learnerfn import get_learner_para



def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        # fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        full_recfldgrn = default_nnpara['full_recfldgrn']
        Info = default_nnpara['Info']
        # para = get_expander_para(nn_name, default_nnpara, embed_size, vocab_tokenizer, init, postprocess)
        para = get_expander_para(full_recfldgrn, Info, embed_size, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '')

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1CDT', 'B-P-EC-A1CV']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1CDT', 'A1CV']
        
        fld_childflds = '&'.join([i for i in childflds])
        # fld_childflds: A1CDT&A1CV
        
        # output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        output_prefix = '-'.join(input_names_nnlvl[0].split('-')[:-1])
        
        # output_prefix: B-P-EC-A1C
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
        
    elif nn_type == 'learner':
        # generate the output name
        assert len(input_names_nnlvl) == 1
        output_name_nnlvl = input_names_nnlvl[0] # just the same name as before
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_learner_para(nn_name, default_nnpara, input_size, output_size, postprocess)
        
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config


In [20]:

def get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   ):
    
    
    # TODO: also add the layer_idx in order to deal with the learn_layer_para.
    # This function also needs to be updated. 
    
    SubUnit_BasicNN_Config_List = []
    
    # print('\n\n************** SubUnit_BasicNN_List ****************')
    # print(SubUnit_BasicNN_List)
    # print(SubUnit_input_names)
    # print(SubUnit_output_name)
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

        # Get the input_names_nnlvl
        if basic_nn_idx == 0:
            input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        else:
            input_names_nnlvl = [output_name_nnlvl]
        
        ##############
        default_nnpara = SubUnit_DefaultBasicNN_List[basic_nn_idx] 
        ##############
        
        
        Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                               input_names_nnlvl, 
                                               default_nnpara, 
                                               embed_size, 
                                               process)
        output_name_nnlvl = Basic_Config['output_name_nnlvl']
        BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
        SubUnit_BasicNN_Config_List.append(BasicNN_Config)
        
        # print('==========================')
        # print(basic_nn_idx, nn_type_nn_name)
        # print(input_names_nnlvl, '<-------- input_names_nnlvl')
        # print(output_name_nnlvl, '<-------- output_name_nnlvl')
        
        
        # also check the input_size of dim and output_size of dim

    final_output_name_nnlvl = output_name_nnlvl

    # if not SubUnit_output_name in final_output_name_nnlvl:
    #     print('xxx errors xxx')
    #     print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
    #     print(SubUnit_output_name, '<------- SubUnit_output_name')
    #     print('xxx errors xxx')
    assert SubUnit_output_name in final_output_name_nnlvl
    # print('\n************** End SubUnit_BasicNN_List ****************')
    return SubUnit_BasicNN_Config_List


In [21]:
len(SubUnit_BasicNN_List)

1

In [22]:
len(SubUnit_DefaultBasicNN_List)

1

In [23]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

s = get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_DefaultBasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    embed_size, 
                                    process, 
                                   )

SubUnit_BasicNN_Config_List = s

In [24]:
SubUnit_BasicNN_Config_List

[{'nn_type_nn_name': 'expander-CateEmbed',
  'Basic_Config': {'input_names_nnlvl': ['B-P-InfoGrn'],
   'output_name_nnlvl': 'B-P-Info',
   'expander_para': {'input_size': None,
    'output_size': 128,
    'nn_type': 'expander',
    'B-P-InfoGrn_tknidx': {'nn_name': 'CateEmbed',
     'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 55}},
    'B-P-InfoGrn_fldidx': {'nn_name': 'CateEmbed',
     'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 5}},
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}},
    'use_wgt': True}}}]

In [25]:
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P..."
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed],"[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '..."


In [26]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

In [27]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

s

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit


,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed],"[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '...","[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [28]:
df_SubUnit['SubUnit_BasicNN_Config_List'].iloc[0]

[{'nn_type_nn_name': 'expander-CateEmbed',
  'Basic_Config': {'input_names_nnlvl': ['B-P-InfoGrn'],
   'output_name_nnlvl': 'B-P-Info',
   'expander_para': {'input_size': None,
    'output_size': 128,
    'nn_type': 'expander',
    'B-P-InfoGrn_tknidx': {'nn_name': 'CateEmbed',
     'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 55}},
    'B-P-InfoGrn_fldidx': {'nn_name': 'CateEmbed',
     'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 5}},
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}},
    'use_wgt': True}}}]

In [29]:
df_SubUnit['SubUnit_BasicNN_Config_List'].iloc[1]

[{'nn_type_nn_name': 'expander-LLMEmbed',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-PNSect-TknzGrn'],
   'output_name_nnlvl': 'B-P-EC-PNSect-Tknz',
   'expander_para': {'input_size': None,
    'output_size': 128,
    'nn_type': 'expander',
    'B-P-EC-PNSect-TknzGrn_tknidx': {'nn_name': 'LLMEmbed',
     'nn_para': {'embedding_size': 128,
      'init': 'bert-base-uncased',
      'tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)}},
    'B-P-EC-PNSect-TknzGrn_fldidx': {'nn_name': 'CateEmbed',
     'nn_para': {'embedding_size': 128, 'init': 'random', 'vocab_size': 3}},
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine':

# SubUnit 

## Module

In [30]:
import torch

from fieldnn.basicnn.expander import Expander_Layer
from fieldnn.basicnn.reducer import Reducer_Layer
from fieldnn.basicnn.merger import Merger_Layer
from fieldnn.basicnn.learner import Learner_Layer

# from ..basicnn.expander import Expander_Layer
# from ..basicnn.reducer import Reducer_Layer
# from ..basicnn.merger import Merger_Layer
# from ..basicnn.learner import Learner_Layer


class SubUnit_Layer(torch.nn.Module):
    '''Currently, it is not latest version'''
    
    def __init__(self, SubUnit_info):
        super(SubUnit_Layer, self).__init__()
        
        # the input names for the SubUnit
        self.SubUnit_input_names = SubUnit_info['input_names']
        
        # the output name for this SubUnit
        self.SubUnit_output_name = SubUnit_info['output_name']
        
        # get the SubUnit's BasicNN Config List
        self.SubUnit_BasicNN_Config_List = SubUnit_info['SubUnit_BasicNN_Config_List']
        
        
        # construct the LayersDict to hold all BasicNN within this SubUnit.
        self.LayersDict = torch.nn.ModuleDict()
        
        # initialize all the BasicNN for this SubUnit.
        for idx, BasicNN_Config_Dict in enumerate(self.SubUnit_BasicNN_Config_List):
            # print(BasicNN_Config_Dict)
            # nn_type_nn_name: like reducer-Max, learner-TFM, merger-Merger, expander-llmembed.
            nn_type_nn_name = BasicNN_Config_Dict['nn_type_nn_name']
            
            # Basic_Config: the Config for this NN.
            Basic_Config = BasicNN_Config_Dict['Basic_Config']
            
            input_names_nnlvl = Basic_Config['input_names_nnlvl']
            output_name_nnlvl = Basic_Config['output_name_nnlvl']
                
            if 'expander' in nn_type_nn_name:
                expander_para = Basic_Config['expander_para']
                # print('\n')
                # print(expander_para)
                # print(input_names_nnlvl)
                # print(output_name_nnlvl)
                assert len(input_names_nnlvl) == 1
                # full_recfldgrn = input_names_nnlvl[0]
                # input_names_nnlvl = [i for i in expander_para if full_recfldgrn in i]
                # input_names_nnlvl = input_names_nnlvl if expander_para['use_wgt'] == False else input_names_nnlvl + [f'{full_recfldgrn}_wgt']
                
                NN = Expander_Layer(input_names_nnlvl, output_name_nnlvl, expander_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            elif 'reducer' in nn_type_nn_name:
                reducer_para = Basic_Config['reducer_para']
                NN = Reducer_Layer(input_names_nnlvl, output_name_nnlvl, reducer_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            elif 'merger' in nn_type_nn_name:
                merger_para = Basic_Config['merger_para']
                NN = Merger_Layer(input_names_nnlvl, output_name_nnlvl, merger_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
                
            elif 'learner' in nn_type_nn_name:
                learner_para = Basic_Config['learner_para']
                NN = Learner_Layer(input_names_nnlvl, output_name_nnlvl, learner_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            else:
                raise ValueError(f'Current BasicNN {nn_type_nn_name} is not available')


    def forward(self, SubUnit_input_names, RECFLD_TO_TENSOR):

        INPUTS_TO_INFODICT = {}
        
        for idx, BasicNN_Config_Dict in enumerate(self.SubUnit_BasicNN_Config_List):
            
            nn_type_nn_name = BasicNN_Config_Dict['nn_type_nn_name']
            Basic_Config = BasicNN_Config_Dict['Basic_Config']
            input_names_nnlvl = Basic_Config['input_names_nnlvl']
            output_name_nnlvl = Basic_Config['output_name_nnlvl']
            
            NN = self.LayersDict[f'{idx}_{nn_type_nn_name}']
            
            # prepare the input.
            if idx == 0:
                for tensor_name in input_names_nnlvl:
                    INPUTS_TO_INFODICT[tensor_name] = RECFLD_TO_TENSOR[tensor_name]
            else:
                for input_name in input_names_nnlvl:
                    assert input_name in INPUTS_TO_INFODICT
                
            output_name_nnlvl, info_dict = NN(input_names_nnlvl, INPUTS_TO_INFODICT)
            
            # current output will be the input in the next round. 
            INPUTS_TO_INFODICT[output_name_nnlvl] = info_dict
            
        # pick up the SubUnit_output_name and its info_dict
        final_output_name_nnlvl = output_name_nnlvl
        SubUnit_output_name = self.SubUnit_output_name
        if not SubUnit_output_name in final_output_name_nnlvl:
            print(f'Mismatched SubUnit Output and Final NN Output: {final_output_name_nnlvl} vs {output_name_nnlvl}')
        
        info_dict = INPUTS_TO_INFODICT[final_output_name_nnlvl]
        
        return SubUnit_output_name, info_dict
    

## Usage

In [31]:
OutputTensor_2_Embed = {}

for idx, SubUnit_info in df_SubUnit.iterrows():
    # print(SubUnit_info)
    # print(SubUnit_info['SubUnit_BasicNN_Config_List'])
    output_name = SubUnit_info['output_name']
    
    print(SubUnit_info)
    
    SubUnitLayer = SubUnit_Layer(SubUnit_info)
    OutputTensor_2_Embed[output_name] = SubUnitLayer
    

SubUnitName                                                                    E
input_names                                                        [B-P-InfoGrn]
output_name                                                             B-P-Info
output_layerid                                                                 3
SubUnit_BasicNN_List                                        [expander-CateEmbed]
SubUnit_DefaultBasicNN_List    [{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P...
SubUnit_BasicNN_Config_List    [{'nn_type_nn_name': 'expander-CateEmbed', 'Ba...
Name: 0, dtype: object
SubUnitName                                                                    E
input_names                                              [B-P-EC-PNSect-TknzGrn]
output_name                                                   B-P-EC-PNSect-Tknz
output_layerid                                                                 5
SubUnit_BasicNN_List                                         [expander-LLMEmbed]
SubUn

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
[i for i in batch_rfg]

['B-P-InfoGrn_wgt',
 'B-P-InfoGrn_tknidx',
 'B-P-InfoGrn_fldidx',
 'B-P-EC-PNSect-TknzGrn_wgt',
 'B-P-EC-PNSect-TknzGrn_tknidx',
 'B-P-EC-PNSect-TknzGrn_fldidx']

In [33]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

In [34]:
OUTPUT_TO_TENSOR = {}

for output_name, SubUnitLayer in OutputTensor_2_Embed.items():
    input_names = SubUnitLayer.SubUnit_input_names
    SubUnit_output_name, info_dict = SubUnitLayer(input_names, RECFLD_TO_TENSOR)
    
    print(SubUnit_output_name)
    print(info_dict['info'].shape)
    OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict

B-P-Info
torch.Size([4, 43, 128])
B-P-EC-PNSect-Tknz
torch.Size([4, 23, 14, 221, 128])


# EmbedBlock

In [35]:
# eventually, we get a df_SubUnit
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed],"[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '...","[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


## Module

In [36]:
import torch
# from .subunit import SubUnit_Layer


class EmbedBlockLayer(torch.nn.Module):
    
    def __init__(self, df_SubUnit):
        super(EmbedBlockLayer, self).__init__()
        self.df_SubUnit = df_SubUnit
        self.SubUnitDict = torch.nn.ModuleDict()
        
        for idx, SubUnit_info in df_SubUnit.iterrows():
            output_name = SubUnit_info['output_name']
            SubUnitLayer = SubUnit_Layer(SubUnit_info)
            self.SubUnitDict[output_name] = SubUnitLayer

    def forward(self, RECFLD_TO_TENSOR):
        
        OUTPUT_TO_TENSOR = {}
        
        for output_name, SubUnitLayer in self.SubUnitDict.items():
            input_names = SubUnitLayer.SubUnit_input_names
            SubUnit_output_name, info_dict = SubUnitLayer(input_names, RECFLD_TO_TENSOR)
            
            assert output_name == SubUnit_output_name
            OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict
        
        return OUTPUT_TO_TENSOR
    

## Usage

In [38]:
EmbedBlock = EmbedBlockLayer(df_SubUnit)

EmbedBlock

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EmbedBlockLayer(
  (SubUnitDict): ModuleDict(
    (B-P-Info): SubUnit_Layer(
      (LayersDict): ModuleDict(
        (0_expander-CateEmbed): Expander_Layer(
          (EmbedDict): ModuleDict(
            (B-P-InfoGrn_tknidx): CateEmbeddingLayer(
              (embedding): Embedding(55, 128, padding_idx=0)
            )
            (B-P-InfoGrn_fldidx): CateEmbeddingLayer(
              (embedding): Embedding(5, 128, padding_idx=0)
            )
          )
          (postprocess): ModuleDict(
            (activator): GELU(approximate='none')
            (dropout): Dropout(p=0.5, inplace=False)
            (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          )
        )
      )
    )
    (B-P-EC-PNSect-Tknz): SubUnit_Layer(
      (LayersDict): ModuleDict(
        (0_expander-LLMEmbed): Expander_Layer(
          (EmbedDict): ModuleDict(
            (B-P-EC-PNSect-TknzGrn_tknidx): LLMEmbeddingLayer(
              (LLM): BertModel(
                (embeddings): Bert

In [39]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

In [ ]:
OUTPUT_TO_TENSOR = EmbedBlock(RECFLD_TO_TENSOR)

In [ ]:
for k, v in OUTPUT_TO_TENSOR.items():
    print(k, v['info'].shape)